In [63]:
import numpy as np
import pandas as pd
import re
import spacy
from textstat.textstat import textstatistics
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from tqdm import tqdm
import requests
import os
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ujjwal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ujjwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ujjwal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [224]:
file = 'output_data/123.0.txt'
with open(file, "r", encoding='utf-8') as read_file:
    reader = read_file.read()

print(reader)

Rise of telemedicine and its Impact on Livelihood by 2040. Telemedicine, the use of technology to diagnose and treat patients remotely, has been rising in recent years. With the advent of high-speed internet and improved video conferencing tools, healthcare providers are increasingly turning to telemedicine to provide care to patients in remote or underserved areas.
Telemedicine, using technology to provide healthcare services remotely, has recently gained popularity. With advancements in communication and medical technology, it has become increasingly possible for doctors and patients to connect and interact from anywhere in the world. This has led to the rise of telemedicine, which has the potential to revolutionize the way healthcare is delivered.#Telemedicine
The increasing focus on preventative healthcare has also driven the rise of telemedicine. As more and more people become aware of the importance of staying healthy, they are looking for ways to prevent illness and maintain the

In [225]:
# Convert to lower
pattern = r'[0-9]'
text = re.sub(pattern,'', reader).lower()

## Avg Sentence length
avg number of words per sentence

In [226]:
def break_into_sentences(text):
    sen = spacy.load('en_core_web_sm')
    doc_text = sen(text)
    return list(doc_text.sents)


In [227]:
def avg_sentence_length(text):
    
    regular_punct = list(string.punctuation)

    sentences_list = break_into_sentences(text)
    word_count = 0
    for sentence in sentences_list:
       list2 = [str(token) for token  in sentence]

       for punc_element in list2:
            for punc in regular_punct:
                  if punc == punc_element:
                        list2.remove(punc_element)

       element_to_remove = '\n'
       list2 = [i for i in list2 if i != element_to_remove]

       word_count += len(list2)

    len_sentences = len(break_into_sentences(text))
    average_sentence_length = (word_count / len_sentences)
    return round(average_sentence_length,2)

avg_sentence_length(text)

# print(f'{avg_sen_len} {word_count}')

20.8

In [228]:
def word_count_raw_text(text):
    
    regular_punct = list(string.punctuation)

    sentences_list = break_into_sentences(text)
    word_list = []
    for sentence in sentences_list:
       list2 = [str(token) for token  in sentence]

       for punc_element in list2:
            for punc in regular_punct:
                  if punc == punc_element:
                        list2.remove(punc_element)

       element_to_remove = '\n'
       list2 = [i for i in list2 if i != element_to_remove]
       
       word_list += list2
    return word_list

word_list= word_count_raw_text(text)
print(len(word_list))

1685


## Processing

In [229]:
# Punctuations

def remove_punctuation(text):
    regular_punct = list(string.punctuation)
    for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, '')
    punc_text = text.strip()
     
    filtered_text = re.sub(r"http\S+", "", punc_text)
    return filtered_text

filter_text = remove_punctuation(text) # processed text

In [230]:
# tokenize
def tokenize(text):
    text_tokenize = word_tokenize(text)
    return text_tokenize
    
tokenized_text = tokenize(filter_text)


## Wordcount

In [231]:
def word_count(tokienized_text):
    lent = len(tokienized_text)
    return lent

text_tokenize = word_count(tokenize(filter_text))
print(text_tokenize)

1676


In [232]:
# Words that are there in word_list but not in the tokenize result. Since the, words are neutral 
# and only 10 in number with no precise meaning ... we will ignore these. And will go with the word 
# count of tokenize 

for k in word_list:
    if k not in tokenized_text:
        print(k) 

high
speed
delivered.#telemedicine
well
being.#healthcare
person
person
alike.#costs
patients.#enhanced
unemployment.#jobs
check
ups
person
 
already
overburdened
high
speed
person
’s
’s
’s
 
g.k.harish
madras


## Avg word length

In [233]:
def avg_word_length(tokenized_text):
  sum_word_length = 0
  for element in tokenized_text:  
    sum_word_length += len(element)
    avg_length = sum_word_length / len(tokenized_text)
  return round(avg_length,2)

avg_word_length(tokenized_text)

5.58

In [234]:
# Stop words
def remove_stop_words(tokenized_text):
    stop_words = set(stopwords.words("english"))
    words = [word for word in tokenized_text if word not in stop_words]
    return words

def lemmatize(words):
    lemmatizer = WordNetLemmatizer()
    words_lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return words_lemmatized

words = remove_stop_words(tokenized_text)
words_lemmatized = lemmatize(words)

### POSITIVE SCORE/ NEGATIVE SCORE / POLARITY SCORE/ SUBJECTIVITY SCORE


In [235]:
def sentiment_analyzer(words, words_lemmatized):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = [sia.polarity_scores(word)['compound'] for word in words_lemmatized]
    average_sentiment = sum(sentiment_score) / len(sentiment_score)
    positive_words = [word for i, word in enumerate(words) if sentiment_score[i] >= 0.05]
    negative_words = [word for i, word in enumerate(words) if sentiment_score[i] <= -0.05]

    total_words = len(words)
    positive_score = (len(positive_words) / total_words) 
    negative_score = (len(negative_words) / total_words)
    subjectivity_score = (positive_score + negative_score)/ ((total_words) + 0.000001) #Range is from 0 to +1
    return round(average_sentiment,2), round(positive_score,2), round(negative_score,2), '%.5f' % subjectivity_score

x, y, z, k = sentiment_analyzer(words, words_lemmatized)
print(k)
print(x)
print(y)
print(z)

0.00014
0.05
0.12
0.02


## Avg Sylable per word

In [236]:
def syllables_count(filter_text):
    return textstatistics().syllable_count(filter_text)
 
# Returns the average number of syllables per
# word in the text
def avg_syllables_per_word(text, tokenized_text):
    syllable = syllables_count(text)
    words = len(tokenized_text)
    avg_syllable = syllable / words
    return round(avg_syllable,2)

avg_syllables_per_word(filter_text,tokenized_text)

1.64

## Complex words

In [241]:
def difficult_words(tokenized_text):
     
    nlp = spacy.load('en_core_web_sm')
    # doc = nlp(text)
 
    # difficult words are those with syllables >= 2
    # easy_word_set is provide by Textstat as
    # a list of common words
    diff_words_set = set()
     
    for word in tokenized_text:
        syllable_count = syllables_count(word)
        if word not in nlp.Defaults.stop_words and syllable_count >= 2:
            diff_words_set.add(word)
 
    return len(diff_words_set), round(((len(diff_words_set)/ len(tokenized_text))*100),2)

total_complex_words, complex_percentage= difficult_words(tokenized_text)

print(total_complex_words)
print(complex_percentage,'%')

132
24.49 %


## Personal Pronouns

In [138]:

def get_pronouns(text):
    find_pronouns = re.compile(r'\b(I|you|he|she|it|we|they|them|him|her|his|hers|its|theirs|our|your(?-i:us))\b',re.I)
    pronouns = find_pronouns.findall(text)

    final_pronouns = ""
    for element in pronouns:
        if element not in final_pronouns:
            final_pronouns = final_pronouns + "/" + element
            
    return final_pronouns

get_pronouns(filter_text)

'/its/they/them/we/our'

## Readability index

In [127]:
def flesch_reading_ease(text):
    """
       Flesch Formula = 206.835 - (1.015 × ASL) - (84.6 × ASW)
        Here,
          ASL = average sentence length (number of words divided by number of sentences)
          ASW = average word length in syllables (number of syllables divided by number of words)
    """
    FRE = 206.835 - (1.015 * avg_sentence_length(text)) - (84.6 * avg_syllables_per_word(text))
    return round(FRE,2)

flesch_reading_ease(text)

46.94

## Functions are ready! Now we have to loop over 112 text files

In [ ]:
break_into_sentences(text) # will work on raw text
avg_sentence_length(text) # will work on raw text
remove_punctuation(text) # will work on raw text > # filtered_text
tokenize(text) # will work on filtered_text > # tokenized_text
word_count(tokienized_text) # will work on tokenized_text
avg_word_length(tokenized_text) # will work on tokenized_text
remove_stop_words(tokenized_text) # will work on tokenized_text > # words
lemmatize(words) # words_lemmatized
sentiment_analyzer(words, words_lemmatized)
syllables_count(text) # will work on filter_text
avg_syllables_per_word(text)  # will work on filter_text
difficult_words(tokenized_text) # will work on tokenized_text
get_pronouns(text) # will work on filter_text
flesch_reading_ease(text) # will work on raw text

## Driver Code

In [260]:
input_url_df = pd.read_csv('input.csv')
raw_df = input_url_df[['URL_ID']]

In [261]:
path = 'output_data/'
raw_temp = raw_df

for file in os.listdir(path):
    print(f'File under progress: {file} ...')
    
    url_id = float(file.split('.txt')[0])
    
    try: 
        with open(os.path.join(path,file), "r", encoding='utf-8') as info_file:
            reader = info_file.read()
        
        pattern = r'[0-9]'
        text = re.sub(pattern,'', reader).lower()

        break_into_sentences(text)
        avg_sent_length = avg_sentence_length(text)

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Avg_sentence_length'] = avg_sent_length
        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Avg_number_of_words_per_sentence'] = avg_sent_length

        filtered_text = remove_punctuation(text)
        tokenized_text = tokenize(filtered_text)
        word_counts = word_count(tokenized_text)

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Word_count'] = int(word_counts)

        avg_word_len = avg_word_length(tokenized_text)

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Avg_word_length'] = int(avg_word_len)

        words = remove_stop_words(tokenized_text)
        words_lemmatized = lemmatize(words)
        polar, pos, neg, sub = sentiment_analyzer(words, words_lemmatized)

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Positive_score'] = pos
        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Negative_score'] = neg
        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Polarity_score'] = polar
        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Subjectivity_score'] = sub

        syllables_count(filtered_text)
        avg_syllable_per_word = avg_syllables_per_word(filtered_text)

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Avg_syllables_per_word'] = avg_syllable_per_word

        complex_count, complex_word_percent = difficult_words(tokenized_text)

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Complex_word_count'] = int(complex_count)
        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Complex_word_percentage'] = complex_word_percent

        pronouns = get_pronouns(text) 

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Personal_pronouns'] = pronouns

        # FRE = flesch_reading_ease(text)
        fog_index = 0.4*(avg_sent_length + complex_word_percent)

        raw_temp.loc[(raw_temp['URL_ID']==url_id), 'Fog_index'] = fog_index

    except Exception as e :
            print(str(e))   

File under progress: 10282.6.txt ...
File under progress: 10744.4.txt ...
File under progress: 11206.2.txt ...
File under progress: 12129.8.txt ...
File under progress: 123.0.txt ...
File under progress: 12591.6.txt ...
File under progress: 13053.4.txt ...
File under progress: 13515.2.txt ...
File under progress: 13977.0.txt ...
File under progress: 14438.8.txt ...
File under progress: 14900.6.txt ...
File under progress: 15362.4.txt ...
File under progress: 15824.2.txt ...
File under progress: 16286.0.txt ...
File under progress: 16747.8.txt ...
File under progress: 17209.6.txt ...
File under progress: 18133.2.txt ...
File under progress: 18595.0.txt ...
File under progress: 19056.8.txt ...
File under progress: 19518.6.txt ...
File under progress: 19980.4.txt ...
File under progress: 20442.2.txt ...
File under progress: 20904.0.txt ...
File under progress: 21365.8.txt ...
File under progress: 21827.6.txt ...
File under progress: 22289.4.txt ...
File under progress: 22751.2.txt ...
Fil

In [262]:
raw_temp

,URL_ID,Avg_sentence_length,Avg_number_of_words_per_sentence,Word_count,Avg_word_length,Positive_score,Negative_score,Polarity_score,Subjectivity_score,Avg_syllables_per_word,Complex_word_count,Complex_word_percentage,Personal_pronouns,Fog_index
0,123.0,20.80,20.80,1676.0,5.0,0.12,0.02,0.05,0.00014,1.64,237.0,14.14,/its/they/them/we/our,13.976
1,321.0,24.38,24.38,617.0,5.0,0.13,0.01,0.05,0.00042,1.72,134.0,21.72,/its/they/them/we/our,18.440
2,2345.0,16.25,16.25,1078.0,5.0,0.08,0.04,0.02,0.00019,1.64,247.0,22.91,/its/they/them/we/our,15.664
3,4321.0,22.82,22.82,1246.0,5.0,0.12,0.03,0.03,0.00020,1.65,324.0,26.00,/its/they/them/we/our,19.528
4,432.0,22.82,22.82,1246.0,5.0,0.12,0.03,0.03,0.00020,1.65,324.0,26.00,/its/they/them/we/our,19.528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,25.11,25.11,658.0,5.0,0.02,0.04,-0.01,0.00015,1.63,172.0,26.14,/its/they/them/we/our,20.500
110,51382.8,33.38,33.38,1676.0,5.0,0.04,0.09,-0.01,0.00012,1.62,341.0,20.35,/its/they/them/we/our,21.492
111,51844.6,26.36,26.36,1781.0,5.0,0.10,0.02,0.03,0.00012,1.60,444.0,24.93,/its/they/them/we/our,20.516
112,52306.4,25.07,25.07,1399.0,4.0,0.06,0.03,0.01,0.00011,1.51,271.0,19.37,/its/they/them/we/our,17.776


In [263]:
output_df = pd.merge(input_url_df, raw_temp, on = 'URL_ID', how = 'outer')

In [264]:
output_df.isna().sum() # for those two url's whose webpage doesn't exit ...values are NaN

URL_ID                              0
URL                                 0
Avg_sentence_length                 2
Avg_number_of_words_per_sentence    2
Word_count                          2
Avg_word_length                     2
Positive_score                      2
Negative_score                      2
Polarity_score                      2
Subjectivity_score                  2
Avg_syllables_per_word              2
Complex_word_count                  2
Complex_word_percentage             2
Personal_pronouns                   2
Fog_index                           2
dtype: int64

In [265]:
output_df.fillna('Not available', inplace = True)

In [266]:
output_df[output_df['Avg_sentence_length'] == 'Not available']

,URL_ID,URL,Avg_sentence_length,Avg_number_of_words_per_sentence,Word_count,Avg_word_length,Positive_score,Negative_score,Polarity_score,Subjectivity_score,Avg_syllables_per_word,Complex_word_count,Complex_word_percentage,Personal_pronouns,Fog_index
24,11668.0,https://insights.blackcoffer.com/how-neural-ne...,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available
37,17671.4,https://insights.blackcoffer.com/covid-19-envi...,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available,Not available


## Convert df to excel file

In [257]:
import openpyxl

In [259]:
output_df.to_excel(r'F:/My_Disk/python/Projects/NLP/outptut.xlsx', index=False)